# Subpockets vs. conformations

Check if our `kissim` subpockets are reasonable across different kinase conformations.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd

from opencadd.databases.klifs import setup_remote
from opencadd.databases.klifs.utils import metadata_to_filepath
from opencadd.structure.pocket import KlifsPocket, PocketViewer

INFO:opencadd.databases.klifs.api:If you want to see an non-truncated version of the DataFrames in this module, use `pd.set_option('display.max_columns', 50)` in your notebook.


## Data retrieval (KLIFS structures)

In [3]:
session = setup_remote()

INFO:opencadd.databases.klifs.api:Set up remote session...
INFO:opencadd.databases.klifs.api:Remote session is ready!


In [4]:
structures = session.structures.all_structures()

Define a function that allows us to get _X_ random structures.

In [5]:
def random_pockets(structures, n_structures, seed, anchor_residues, subpocket_names, subpocket_colors, klifs_session):
    
    random_structures = structures.sample(n=n_structures, random_state=seed)
    subpockets = {
        "anchor_residue.klifs_ids": anchor_residues,
        "subpocket.name": subpocket_names,
        "subpocket.color": subpocket_colors
    }
    pockets = []
    for structure_klifs_id in random_structures["structure.klifs_id"]:
        print(structure_klifs_id)
        pocket = KlifsPocket.from_structure_klifs_id(structure_klifs_id, subpockets, klifs_session=klifs_session)
        pockets.append(pocket)
        
    return pockets

In [6]:
N_STRUCTURES = 3
SEED = 1
ANCHOR_RESIDUES = [[16, 47, 80], [19, 24, 81], [6, 48, 75]]
SUBPOCKET_NAMES = ["hinge_region", "dfg_region", "front_pocket"]

## Kinase conformations [WIP]

### DFG conformations

In [7]:
pockets_by_conformation = {
    "in": random_pockets(
        structures[(structures["structure.dfg"] == "in")], 
        N_STRUCTURES, 
        SEED, 
        ANCHOR_RESIDUES, 
        SUBPOCKET_NAMES, 
        ["magenta", "cyan", "green"], 
        session),
    "out": random_pockets(
        structures[(structures["structure.dfg"] == "out")], 
        N_STRUCTURES, 
        SEED, 
        ANCHOR_RESIDUES, 
        SUBPOCKET_NAMES, 
        ["violet", "turquoise", "lime"], 
        session
    ),
}

6335


INFO:opencadd.structure.pocket.core:Pocket 6335 (set pocket residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [('_', '84'), ('_', '85')]
INFO:opencadd.structure.pocket.core:Pocket 6335 (set region residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [(None, '84'), (None, '85')]


13036


INFO:opencadd.structure.pocket.core:Pocket 13036 (set pocket residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [('_', '51')]
INFO:opencadd.structure.pocket.core:Pocket 13036 (set region residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [(None, '51')]


1670


INFO:opencadd.structure.pocket.core:Pocket 1670 (set pocket residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [('_', '7'), ('_', '8'), ('_', '9')]
INFO:opencadd.structure.pocket.core:Pocket 1670 (set region residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [(None, '7'), (None, '8'), (None, '9')]


5730
5028


INFO:opencadd.structure.pocket.core:Pocket 5028 (set pocket residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [('_', '50')]
INFO:opencadd.structure.pocket.core:Pocket 5028 (set region residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [(None, '50')]


11439


INFO:opencadd.structure.pocket.core:Pocket 11439 (set pocket residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [('_', '83'), ('_', '84'), ('_', '85')]
INFO:opencadd.structure.pocket.core:Pocket 11439 (set region residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [(None, '84'), (None, '85')]
INFO:opencadd.structure.pocket.core:Pocket 11439 (set region residues): The following input residues were dropped because they cannot be cast to an integer (residue PDB ID, residue index): [(None, '83')]


In [8]:
viewer = PocketViewer()
for _, pockets in pockets_by_conformation.items():
    for pocket in pockets:
        viewer.add_pocket(pocket, sphere_opacity=1)
viewer.viewer

NGLWidget()

In [9]:
viewer.structure_names

[6335, 13036, 1670, 5730, 5028, 11439]

In [ ]:
viewer.show_only(5730)

## Check aC-helix conformation

__KLIFS definitions__

* aC helix conformation: Distance between the Ca atoms of residue 82 (DFG) and conserved residue 24 (aC helix).
  * aC-in: $[4, 7.2]$
  * aC-out: $>9.3$
  * aC-out-like: $[7.2, 9.3]$
* DFG conformation: Manually.

__Examples by Ung _et al.___

BRAF structures in different conformations taken from [Ung _et al._ (2018)](https://www.sciencedirect.com/science/article/pii/S2451945618301491).

|             | aC-in        | aC-out       |
|-------------|--------------|--------------|
| __DFG-in__  | 2FB8 _CIDI_  | 4EHG _CODI_  |
| __DFG-out__ | 1UWH _CIDO_  | 4XV9 _CODO_  |

_Note_: KLIFS conformation annotation differs from Ung _et al._.